In [1]:
# Notes: 
# 1. You need to put the covariance matrix csv files in this folder
# 2. Some of the cells can be a bit computationally expensive, hence I save the output 
#    as pickles to be easily reloaded

In [4]:
from validphys.calcutils import calc_chi2
import numpy as np
import pandas as pd
from validphys.core import ExperimentSpec, FKTableSpec
from validphys import results
from validphys.loader import Loader
from validphys.api import API
import scipy.linalg as la

# Reading in deuteron covmats
it0 = pd.read_csv(
    "covmatrix_global_proton.csv",
    dtype={"user_id": float},
    index_col=[0,1,2], header=[0,1,2]
)

it1dw = pd.read_csv(
    "covmatrix_ite.csv",
    dtype={"user_id": float},
    index_col=[0,1,2], header=[0,1,2]
)

it1shift = pd.read_csv(
    "covmatrix_shift_ite_1.csv",
    dtype={"user_id": float},
    index_col=[0,1,2], header=[0,1,2]
)

In [5]:
# Renaming experiment DYE886R -> DYE886 to correct covariance matrix.
# Commented out because it was changed manually in the csv file.
#tups = []
# for tup in it0.index:
#     if tup[0] == "DYE886R":
#         newtup = ("DYE886", tup[1], tup[2])
#     else: newtup = tup
#     tups.append(newtup)
# newindex = pd.MultiIndex.from_tuples(tups, names=("experiment", "dataset", "id"))

# Relabelling dataframes because otherwise column index is a string rather than an int and this causes problems
# down the line
it0 = pd.DataFrame(it0.values, index=it0.index, columns=it0.index)
it1dw = pd.DataFrame(it1dw.values, index=it1dw.index, columns=it1dw.index)
it1shift = pd.DataFrame(it1shift.values, index=it1shift.index, columns=it1shift.index)

In [6]:
# Importing dataset index so we can wrap everything in a dataframe to prevent misalignment
dsindex_bl = API.experiments_index(experiments={"from_": "fit"},
                                   fit="200609-ern-001",
                                   theoryid=53,
                                   use_cuts="fromfit",
                                   pdf={"from_": "fit"})

In [4]:
# Getting D and T to calculate diffs
datth_bl = API.experiments_results(experiments={"from_": "fit"},
                                   fit="200609-ern-001",
                                   theoryid=53,
                                   use_cuts="fromfit",
                                   pdf={"from_": "fit"})

In [25]:
diffs = []
for experiment in datth_bl:
    diffs.append(experiment[0].central_value - experiment[1].central_value)
diffs_bl = pd.DataFrame([item for sublist in diffs for item in sublist], index=dsindex_bl)


In [28]:
datth_it0 = API.experiments_results(experiments={"from_": "fit"},
                            fit="NNPDF31_nnlo_as_0118_global_deut",
                            theoryid=53,
                            use_cuts="fromfit",
                            pdf={"from_": "fit"})

In [29]:
diffs = []
for experiment in datth_it0:
    diffs.append(experiment[0].central_value - experiment[1].central_value)
diffs_it0 =pd.DataFrame([item for sublist in diffs for item in sublist], index=dsindex_bl)


In [9]:
datth_it1dw = API.experiments_results(experiments={"from_": "fit"},
                                     fit="NNPDF31_nnlo_as_0118_global_deut_ite",
                                     theoryid=53,
                                     use_cuts="fromfit",
                                     pdf={"from_": "fit"})

In [12]:
diffs = []
for experiment in datth_it1dw:
    diffs.append(experiment[0].central_value - experiment[1].central_value)
diffs_it1dw =pd.DataFrame([item for sublist in diffs for item in sublist], index=dsindex_bl)


In [4]:
datth_it1shifted = API.experiments_results(experiments={"from_": "fit"},
                                     fit="NNPDF31_nnlo_as_0118_global_deut_ite_shift",
                                     theoryid=53,
                                     use_cuts="fromfit",
                                     pdf={"from_": "fit"})

In [5]:
diffs = []
for experiment in datth_it1shifted:
    diffs.append(experiment[0].central_value - experiment[1].central_value)
diffs_it1shifted =pd.DataFrame([item for sublist in diffs for item in sublist], index=dsindex_bl)


In [7]:
# Pickling items for easy reloading - from now on can skip cells 5-13
import pickle
pickle.dump( diffs_bl, open( "diffs_bl.p", "wb" ) )
pickle.dump( diffs_it0, open( "diffs_it0.p", "wb" ) )
pickle.dump( diffs_it1dw, open( "diffs_it1dw.p", "wb" ) )
pickle.dump( diffs_it1shifted, open( "diffs_it1shifted.p", "wb" ) )

NameError: name 'diffs_bl' is not defined

In [8]:
# Reloading pickles
diffs_bl = pd.read_pickle("diffs_bl.p")
diffs_it0 = pd.read_pickle("diffs_it0.p")
diffs_it1dw = pd.read_pickle("diffs_it1dw.p")
diffs_it1shifted = pd.read_pickle("diffs_it1shifted.p")

In [9]:
#diffs_it0 = pd.DataFrame(diffs_it0.values, index=C_orig.index)
#diffs_bl = pd.DataFrame(diffs_bl.values, index=newindex)
#diffs_it1dw = pd.DataFrame(diffs_it1dw.values, index=newindex)
#diffs_it1shifted = pd.DataFrame(diffs_it1shifted.values, index=newindex)

In [9]:
# Loading original covmat (experimental)
C_orig = API.experiments_covmat( experiments={"from_": "fit"},
                                   fit ="200609-ern-001",
                                   theoryid=53,
                                   use_cuts="fromfit",
                                   pdf={"from_": "fit"})

/home/s1303034/miniconda3/envs/nnpdf-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/s1303034/miniconda3/envs/nnpdf-dev/lib/python3.7/site-packages/reportengine/api.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self.__call__(name, **kwargs)


In [10]:
# List of datasets in orig covmat and in deuteron fit
dslist = list(dict.fromkeys([tup[1] for tup in C_orig.index]))
dslist_small = list(dict.fromkeys([tup[1] for tup in it0.index]))

In [11]:
# Function to extend the dimensions of a small covmat to that of a big covmat, filling in the empty entries with 0s
def extend_covmat(dslist, bigcovmat, smallcovmat):
    # Make dimensions match those of exp covmat. First make empty df of
    # exp covmat dimensions
    empty_df = pd.DataFrame(0, index=C_orig.index, columns=C_orig.index)
    covmats = []
    for ds1 in dslist:
        for ds2 in dslist:
            if (ds1 in smallcovmat.index.unique(level=1)) and (ds2 in smallcovmat.index.unique(level=1)):
                # If both datasets are in the small covmat, take the relevant ds covmat out the small covmat
                covmat = smallcovmat.xs(ds1,level=1, drop_level=False).T.xs(ds2, level=1, drop_level=False).T
            else:
                # If not, make a ds covmat of 0s of the relevant dimensions 
                covmat = empty_df.xs(ds1,level=1, drop_level=False).T.xs(ds2, level=1, drop_level=False).T
            covmat.reset_index()
            # covmats is a list of the ds covmats in order
            covmats.append(covmat)
    # Chunks is a list of lists of covmats, one list of covmats for each dataset
    chunks = []
    for x in range(0, len(covmats), len(dslist)):
        chunk = covmats[x:x+len(dslist)]
        chunks.append(chunk)
    # Concatenate each chunk into a strip so we have N_dataset strips of covmat
    strips = []
    i=0
    for chunk in chunks:
        i=i+1
        strip = pd.concat(chunk, axis=1)
        strips.append(strip.T)
    strips.reverse()
    # Stack the strips to construct the full covmat
    full_df = pd.concat(strips, axis=1)
    full_df = full_df.reindex(bigcovmat.index)
    full_df = ((full_df.T).reindex(bigcovmat.index)).T
    return full_df

In [12]:
# Extending the deuteron covmats to match size of C_orig
it0total = extend_covmat(dslist, C_orig, it0)
it1dwtotal = extend_covmat(dslist, C_orig, it1dw)
it1shifttotal = extend_covmat(dslist, C_orig, it1shift)

In [13]:
# Calculating total chi2s
calc_chi2(la.cholesky(C_orig, lower=True), diffs_bl)/len(diffs_bl)

array([1.17868292])

In [14]:
calc_chi2(la.cholesky(it0total+C_orig, lower=True), diffs_it0)/len(diffs_it0)

array([1.16000901])

In [15]:
calc_chi2(la.cholesky(it1dwtotal+C_orig, lower=True), diffs_it1dw)/len(diffs_it1dw)

array([1.15869441])

In [16]:
calc_chi2(la.cholesky(it1shifttotal+C_orig, lower=True), diffs_it1shifted)/len(diffs_it1shifted)

array([1.16601756])

In [17]:
# Function to return chi2s by dataset
def chi2s_by_dataset(covmat, diffs):
    chi2s = []
    for dataset in dslist:
        dscovmat = covmat.xs(dataset,level=1, drop_level=False).T.xs(dataset, level=1, drop_level=False).T
        dsdiffs = diffs.xs(dataset, level=1, drop_level=False)
        chi2 = calc_chi2(la.cholesky(dscovmat, lower=True), dsdiffs)/len(dsdiffs)
        chi2s.append((dataset, chi2[0]))
    chi2table = pd.DataFrame(chi2s, columns=["dataset", "chi2"])
    return chi2table
        

In [22]:
#chi2s_by_dataset(C_orig+it1dwtotal, diffs_it1dw)

In [36]:
exceptions = {"NMC":["NMC", "NMCPD"], "SLAC": ["SLACP", "SLACD"], "BCDMS":["BCDMSP", "BCDMSD"]}
explist = list(dict.fromkeys([tup[0] for tup in C_orig.index]))

In [56]:
def chi2s_by_custom_grouping(covmat, diffs, exceptions):
    chi2s = []
    index = []
    for exp in explist:
        if exp in exceptions:
            datasets = exceptions[exp]
            for dataset in datasets:
                dscovmat = covmat.xs(dataset,level=1, drop_level=False).T.xs(dataset, level=1, drop_level=False).T
                dsdiffs = diffs.xs(dataset, level=1, drop_level=False)
                chi2 = calc_chi2(la.cholesky(dscovmat, lower=True), dsdiffs)/len(dsdiffs)
                index.append(dataset)
                chi2s.append(chi2[0])
        else:
            dscovmat = covmat.xs(exp,level=0, drop_level=False).T.xs(exp, level=0, drop_level=False).T
            dsdiffs = diffs.xs(exp, level=0, drop_level=False)
            chi2 = calc_chi2(la.cholesky(dscovmat, lower=True), dsdiffs)/len(dsdiffs)
            index.append(exp)
            chi2s.append(chi2[0])
    chi2table = pd.DataFrame(chi2s, index=index, columns=[r"$\chi^2$"])
    return chi2table

In [58]:
blchi2 = chi2s_by_custom_grouping(C_orig, diffs_bl, exceptions)
print(blchi2.to_latex())

In [ ]:
it0chi2 = chi2s_by_custom_grouping(C_orig, diffs_bl, exceptions)